## Repliikide lausestamine test5

In [1]:
from estnltk import Text
import sqlite3
import pickle
from estnltk_core.layer_operations import split_by_sentences

Tühi järjend repliikide hoiustamiseks:

In [18]:
corpus = []

Andmebaasi tabelist loetakse sisse repliikidele vastavad Text-objektid:

In [19]:
con = sqlite3.connect("data.db")

In [20]:
cur = con.cursor()

In [21]:
for row in cur.execute("SELECT line FROM data"):
    row = pickle.loads(row[0])
    corpus.append(row)

In [22]:
con.close()

In [23]:
len(corpus)

391

In [24]:
test_text = corpus[5]
display(test_text)

Text(text='Aga ma ei tea kas kaitsejõud hakkasid seletama et ei et see mingi raskeveok koos nende soomukite kes saadab sellised me lihtsalt kontrollime tammel üldse olemas on nüüd ajame korraga siis välja ka sõjaosalejatega rohkematele kommenteerib aga minu punkte siin ette et.\nMõtteks nii nagu Andrus Ansipil on selline lauset alustuse stamm fraas nagu et ma ei saa olla nõus nendega kes mõtles peaminister ma ei saa olla nõus nendega kes väidavad et nad suudavad inflatsiooni järgmise aasta esimesel poolel kuidagi kontrollijaid.\nSest vaadakem üks asi on käia aga olgu käes ja vaadata kas see.\nViisteist koma kuuskümmend neliikkagi kuidas kuidas need on eks ole soomlasel Eestis midagi osta ka teine asi on näha üks ühele võrrelda Vassel kes eurokeeles euro alus neid hindu näiteks üks mu tuttav intellektuaal Soomest ütleb et sealsed inimesed kevad Eestis nahkköitesse panema sama.\nNo kui sul on mingi armsa ja kirjatöid ja seal või või.\nKui raamat perekonna mälestused ja nahkköites ei paneks selle jutu järgmisele kontrolli Natanzis.\nHelsingis maksab neli Gent kaheksa ei uuri üks suur paks üks kõige ja meil maksab siis umbes nagu seitse juure kaheksakümmend näiteks talikummid üle suvehoidmine või näiteks üks tund.\nTa meelelahutus tegelema ütleb et kontserdi korraldab esinenud ärihelivõimenduse ja kõige selle mehe kaasa aga ma ülevedamise eurodes end lihtsalt kordades vahe noh advokaadi tunni seal kaks pool kolm korda kui kui samu asju tuleb arutada näiteks sadamas kuidas.\nSadamakaupa kus tolliterminaalist see hindade vahel lihtsalt peab tekitama olukorra kus need kes suudavad soomlasele midagi müüa teenivad teised siis ei suuda nende kõrgete hindadega eriti Tallinna hindadega väljakult väljatuleb inimesed kaasas selliselt tekib ma ei saa olla nõus nendega kes.')

In [25]:
test_text.ner

Layer(name='ner', attributes=('nertag',), spans=SL[EnvelopingSpan(['Andrus', 'Ansipil'], [{'nertag': 'PER'}]),
EnvelopingSpan(['Eestis'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Vassel'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Soomest'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Eestis'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Natanzis'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Helsingis'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Tallinna'], [{'nertag': 'LOC'}])])

Kuna repliigid on suhteliselt kirjakeelsed, lausestan esialgu split_by_sentences abil:

In [26]:
line_sentences = split_by_sentences(text=test_text, 
                           layers_to_keep=list(test_text.layers),
                           trim_overlapping=True
                           )

In [27]:
for sentence in line_sentences:
    if 'PER' in sentence.ner.nertag:
        display(sentence)
        display(sentence.ner)

Text(text='Mõtteks nii nagu Andrus Ansipil on selline lauset alustuse stamm fraas nagu et ma ei saa olla nõus nendega kes mõtles peaminister ma ei saa olla nõus nendega kes väidavad et nad suudavad inflatsiooni järgmise aasta esimesel poolel kuidagi kontrollijaid.')

Layer(name='ner', attributes=('nertag',), spans=SL[EnvelopingSpan(['Andrus', 'Ansipil'], [{'nertag': 'PER'}])])

In [28]:
for sentence in line_sentences:
    sentence.meta['line_number'] = test_text.meta['line_number']
    sentence.meta['person'] = test_text.meta['person']
    sentence.meta['date'] = test_text.meta['date']
display(sentence.meta)

{'line_number': 6, 'person': 'Tundmatu1', 'date': datetime.date(2010, 10, 30)}

Andmebaasi tabelisse lisatakse uus veerg repliikides olevate lausete jaoks, mis lisatakse ridadesse järjenditena. Sealjuures lisatakse igale lausele vastava repliigi metaandmed.

In [8]:
con = sqlite3.connect("data.db")
cur = con.cursor()
cur.execute('''ALTER TABLE data ADD COLUMN sentences BLOB''')

In [9]:
for text in corpus:
    line_sentences = split_by_sentences(text=text, 
                           layers_to_keep=list(text.layers),
                           trim_overlapping=True
                           )
    for sentence in line_sentences:
        sentence.meta['line_number'] = text.meta['line_number']
        sentence.meta['person'] = text.meta['person']
        sentence.meta['date'] = text.meta['date']
    sen_bytes = pickle.dumps(line_sentences)
    cur.execute("""UPDATE data SET sentences = ? WHERE number = ? AND person = ? AND date = ?""", (bytearray(sen_bytes), line_sentences[0].meta['line_number'], line_sentences[0].meta['person'], line_sentences[0].meta['date']))
    con.commit()

In [10]:
con.close()